# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Шаги исследования

1. Подготовка данных
2. Обучение моделей
3. Анализ моделей

## 1. Подготовка данных

In [1]:
!pip install -U scikit-learn

In [2]:
# Импорт нужных библиотек
import pandas as pd
import numpy as np
import math

import lightgbm as lgb

from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (
    GridSearchCV, 
    RandomizedSearchCV,
    train_test_split
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)

RANDOM_STATE = 12345

# Категориальные признаки, которые нужно закодировать 
ohe_features = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'repaired', 'registrationmonth', 'registrationyear']
# Количественные признаки, которые нужно масштабировать 
num_features = ['power', 'kilometer']

In [3]:
# Загрузим данные
data = pd.read_csv('/datasets/autos.csv')

In [4]:
def about_data(data):
    display(data.sample(5))
    display(data.info())
    display(data.describe(include='all'))

In [5]:
# Изучим данные
about_data(data)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
347149,2016-04-01 11:59:01,1500,wagon,2003,manual,75,NaN,150000,5,petrol,peugeot,NaN,2016-04-01 00:00:00,0,47229,2016-04-07 08:46:07
70291,2016-03-18 07:53:30,2350,NaN,2016,manual,68,aygo,150000,5,petrol,toyota,no,2016-03-18 00:00:00,0,58849,2016-03-18 07:53:30
728,2016-03-20 11:53:15,5500,sedan,2003,auto,163,a4,150000,11,petrol,audi,no,2016-03-20 00:00:00,0,97450,2016-03-26 20:16:52
340568,2016-04-03 08:50:39,399,small,1995,manual,75,NaN,150000,0,NaN,volkswagen,NaN,2016-04-03 00:00:00,0,47803,2016-04-07 10:16:15
36248,2016-03-24 19:59:45,2200,sedan,2001,manual,129,c_klasse,150000,5,petrol,mercedes_benz,no,2016-03-24 00:00:00,0,40476,2016-03-29 06:45:24


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
count,354369,354369.000000,316879,354369.000000,334536,354369.000000,334664,354369.000000,354369.000000,321474,354369,283215,354369,354369.0,354369.000000,354369
unique,271174,NaN,8,NaN,2,NaN,250,NaN,NaN,7,40,2,109,NaN,NaN,179150
top,2016-03-24 14:49:47,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,NaN,91457,NaN,268251,NaN,29232,NaN,NaN,216352,77013,247161,13719,NaN,NaN,17
mean,NaN,4416.656776,NaN,2004.234448,NaN,110.094337,NaN,128211.172535,5.714645,NaN,NaN,NaN,NaN,0.0,50508.689087,NaN
std,NaN,4514.158514,NaN,90.227958,NaN,189.850405,NaN,37905.341530,3.726421,NaN,NaN,NaN,NaN,0.0,25783.096248,NaN
min,NaN,0.000000,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,1050.000000,NaN,1999.000000,NaN,69.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30165.000000,NaN
50%,NaN,2700.000000,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49413.000000,NaN
75%,NaN,6400.000000,NaN,2008.000000,NaN,143.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71083.000000,NaN


In [6]:
# Приведем названия столбцов к нижнему регистру
data.columns = map(str.lower, data.columns)
data.columns

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'kilometer', 'registrationmonth', 'fueltype', 'brand',
       'repaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [7]:
# Удалим признаки, которые не являются информативными
data = data.drop(['datecrawled', 'datecreated', 'postalcode', 'numberofpictures', 'lastseen'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   price              354369 non-null  int64 
 1   vehicletype        316879 non-null  object
 2   registrationyear   354369 non-null  int64 
 3   gearbox            334536 non-null  object
 4   power              354369 non-null  int64 
 5   model              334664 non-null  object
 6   kilometer          354369 non-null  int64 
 7   registrationmonth  354369 non-null  int64 
 8   fueltype           321474 non-null  object
 9   brand              354369 non-null  object
 10  repaired           283215 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


Колонки `datecrawled`, `datecreated`, `postalcode`, `lastseen` неинформативны, так как не несут в себе никаких характеристик, способных влиять на таргет - колонку `price`. Колонка `numberofpictures` полностью нулевая.

In [8]:
# Посчитаем процент пропусков в каждом столбце
print((data.isna().sum() / data.shape[0]) * 100)

price                 0.000000
vehicletype          10.579368
registrationyear      0.000000
gearbox               5.596709
power                 0.000000
model                 5.560588
kilometer             0.000000
registrationmonth     0.000000
fueltype              9.282697
brand                 0.000000
repaired             20.079070
dtype: float64


In [9]:
# Заполним пропуски с помощью заглушек
columns_to_fill = ['vehicletype', 'gearbox', 'model', 'fueltype', 'repaired']
for column in columns_to_fill:
    data[column] = data[column].fillna('no')
print((data.isna().sum() / data.shape[0]) * 100)

price                0.0
vehicletype          0.0
registrationyear     0.0
gearbox              0.0
power                0.0
model                0.0
kilometer            0.0
registrationmonth    0.0
fueltype             0.0
brand                0.0
repaired             0.0
dtype: float64


Так как природу пропусков определить представляется маловозможным, поставим заглушки в данных признаках.

In [10]:
data.duplicated().sum()

30890

In [11]:
# Удалим дубликаты
data = data.drop_duplicates()
data.shape

(323479, 11)

In [12]:
display(data.describe())

,price,registrationyear,power,kilometer,registrationmonth
count,323479.000000,323479.000000,323479.000000,323479.000000,323479.000000
mean,4408.007487,2004.252159,110.267646,128068.947289,5.693433
std,4523.118417,91.571014,196.814871,38009.696049,3.724624
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1000.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6390.000000,2008.000000,141.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


In [13]:
# Напишем функцию для обработки аномальных значений
def function_1(df, columns):
    for col in columns:
        q1 = df[col].quantile(q=0.25)
        q3 = df[col].quantile(q=0.75)
        iqr = q3-q1
        df = df[df[col] != 0]
        df = df.loc[df[col] > (q1 - 1.5*iqr)]
        df = df.loc[df[col] < (q3 + 1.5*iqr)]
    return df    

In [14]:
# Колонку 'price' обработаем отдельно
iqr_price = data['price'].quantile(q=0.75) - data['price'].quantile(q=0.25)
data = data.loc[data['price'] > (data['price'].quantile(q=0.25) - 1.5*iqr_price)]
data = data.loc[data['price'] != 0]

In [15]:
df = function_1(data, ['registrationyear', 'power', 'kilometer'])

In [16]:
df

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired
1,18300,coupe,2011,manual,190,no,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
...,...,...,...,...,...,...,...,...,...,...,...
354361,5250,no,2016,auto,150,159,150000,12,no,alfa_romeo,no
354362,3200,sedan,2004,manual,225,leon,150000,5,petrol,seat,yes
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


**Вывод**:
- Выгрузили и ознакомились с данными;
- Удалили неинформативные признаки: `datecrawled`, `datecreated`, `postalcode`, `numberofpictures`, `lastseen`;
- Заполнили пропуски в столбцах `vehicletype`, `gearbox`, `model`, `fueltype`, `repaired` с помощью заглушек;
- Удалили дубликаты;
- Избавились от аномальных значений.

## 2. Обучение моделей

In [17]:
# Разделим выборку на фичи и таргеты
features = df.drop('price', axis=1)
target = df['price']

In [18]:
# Изменим тип данных для столбцов *registrationyear* и *registrationmonth*
for i in ['registrationyear', 'registrationmonth']:
    df[i] = df[i].astype('str')

In [19]:
# Разделим выборки на обучающую, валидационную и тестовую
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                              test_size=0.25, random_state=RANDOM_STATE)
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, 
                                                                              test_size=0.5, random_state=RANDOM_STATE)

In [20]:
features_train_ridge = features_train.copy()
features_test_ridge = features_test.copy()
features_valid_ridge = features_valid.copy()

In [21]:
# признак repaired уже бинарный, его не будем кодировать/масштабировать
# добавляем remainder='passthrough', чтобы он не пропал 
col_transformer_ridge = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        ohe_features
    ),
    (
        StandardScaler(), 
        num_features
    ),
    remainder='passthrough',
    verbose_feature_names_out=False
)


features_train_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.fit_transform(features_train_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)


features_train_ridge.shape

(172324, 348)

In [22]:
# трансформируем валидационную выборку
features_valid_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.transform(features_valid_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)


features_valid_ridge.shape

(28721, 348)

In [23]:
# Построим модель линейной регрессии и проверим на валидационной выборке
model = LinearRegression()
%time model.fit(features_train_ridge, target_train)
%time predicted = model.predict(features_valid_ridge)
rmse_0 = math.sqrt(mean_squared_error(target_valid, predicted))
print(rmse_0)

CPU times: user 18.7 s, sys: 24.5 s, total: 43.2 s
Wall time: 43.3 s
CPU times: user 73.3 ms, sys: 73.6 ms, total: 147 ms
Wall time: 138 ms
1808.2522195169809


Значение метрики RMSE для **линейной регрессии** получилось равным 1808.25, время обучения на валидационной выборке -  43.3 s, предсказания: 138 ms.

In [27]:
%%time
# Построим модель градиентного бустинга с перебором гиперпараметров

parametrs = {'n_estimators': range(20, 30, 4), 
             'max_depth': range(5, 9, 2)}

booster = lgb.LGBMModel(objective="regression", score='rmse', return_train_score=True)

grid = GridSearchCV(booster, parametrs, cv=5)

grid.fit(features_train_ridge, target_train, eval_set=[(features_valid_ridge, target_valid),], eval_metric="rmse", early_stopping_rounds=10)

print("Лучшее значение глубины и количества деревьев:", grid.best_estimator_)
print("Лучшее значение RMSE:", grid.best_score_)

/opt/conda/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: score
[LightGBM] [Warning] Unknown parameter: return_train_score
[1]	valid_0's rmse: 3690.7	valid_0's l2: 1.36213e+07
[2]	valid_0's rmse: 3502.78	valid_0's l2: 1.22695e+07
[3]	valid_0's rmse: 3341.01	valid_0's l2: 1.11624e+07
[4]	valid_0's rmse: 3200.78	valid_0's l2: 1.0245e+07
[5]	valid_0's rmse: 3080.02	valid_0's l2: 9.48653e+06
[6]	valid_0's rmse: 2976.46	valid_0's l2: 8.85933e+06
[7]	valid_0's rmse: 2875.85	valid_0's l2: 8.2705e+06
[8]	valid_0's rmse: 2789.65	valid_0's l2: 7.78215e+06
[9]	valid_0's rmse: 2709.66	valid_0's l2: 7.34228e+06
[10]	valid_0's rmse: 2640.49	valid_0's l2: 6.97217e+06
[11]	valid_0's rmse: 2579.54	valid_0's l2: 6.65403e+06
[12]	valid_0's rmse: 2523.79	valid_0's l2: 6.36949e+06
[13]	valid_0's rmse: 2472.79	valid_0's l2: 6.1147e+06
[14]	valid_0's rmse: 2429.51	valid_0's l2: 5.9025e+06
[15]	valid_0's rmse: 2386.37	valid_0's l2: 5.69476e+06
[16]	valid_0's rmse: 2350.1	valid_0's l2: 5.52298e+06
[17]	valid_0's rmse: 2315.45	v

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
TypeError: 'str' object is not callable

  warnings.warn(


[1]	valid_0's rmse: 3691.27	valid_0's l2: 1.36254e+07
[2]	valid_0's rmse: 3502.69	valid_0's l2: 1.22688e+07
[3]	valid_0's rmse: 3341.16	valid_0's l2: 1.11633e+07
[4]	valid_0's rmse: 3200.48	valid_0's l2: 1.02431e+07
[5]	valid_0's rmse: 3080.25	valid_0's l2: 9.48796e+06
[6]	valid_0's rmse: 2976.15	valid_0's l2: 8.85749e+06
[7]	valid_0's rmse: 2875.19	valid_0's l2: 8.26674e+06
[8]	valid_0's rmse: 2788.27	valid_0's l2: 7.77446e+06
[9]	valid_0's rmse: 2708.02	valid_0's l2: 7.33339e+06
[10]	valid_0's rmse: 2639.2	valid_0's l2: 6.96539e+06
[11]	valid_0's rmse: 2576.32	valid_0's l2: 6.63744e+06
[12]	valid_0's rmse: 2519.61	valid_0's l2: 6.34844e+06
[13]	valid_0's rmse: 2472.58	valid_0's l2: 6.11366e+06
[14]	valid_0's rmse: 2424.72	valid_0's l2: 5.87926e+06
[15]	valid_0's rmse: 2382.21	valid_0's l2: 5.67493e+06
[16]	valid_0's rmse: 2346.17	valid_0's l2: 5.50453e+06
[17]	valid_0's rmse: 2313.49	valid_0's l2: 5.35221e+06
[18]	valid_0's rmse: 2283.02	valid_0's l2: 5.21217e+06
[19]	valid_0's rmse:

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


[1]	valid_0's rmse: 3690.9	valid_0's l2: 1.36227e+07
[2]	valid_0's rmse: 3502.99	valid_0's l2: 1.22709e+07
[3]	valid_0's rmse: 3341.09	valid_0's l2: 1.11629e+07
[4]	valid_0's rmse: 3201.59	valid_0's l2: 1.02502e+07
[5]	valid_0's rmse: 3080.69	valid_0's l2: 9.49064e+06
[6]	valid_0's rmse: 2976.74	valid_0's l2: 8.86097e+06
[7]	valid_0's rmse: 2874.2	valid_0's l2: 8.26102e+06
[8]	valid_0's rmse: 2788.06	valid_0's l2: 7.77327e+06
[9]	valid_0's rmse: 2708.7	valid_0's l2: 7.33708e+06
[10]	valid_0's rmse: 2638.95	valid_0's l2: 6.96404e+06
[11]	valid_0's rmse: 2578.44	valid_0's l2: 6.64836e+06
[12]	valid_0's rmse: 2521.78	valid_0's l2: 6.35936e+06
[13]	valid_0's rmse: 2472.69	valid_0's l2: 6.11419e+06
[14]	valid_0's rmse: 2426.36	valid_0's l2: 5.88725e+06
[15]	valid_0's rmse: 2386.4	valid_0's l2: 5.69491e+06
[16]	valid_0's rmse: 2349.1	valid_0's l2: 5.51828e+06
[17]	valid_0's rmse: 2315.59	valid_0's l2: 5.36197e+06
[18]	valid_0's rmse: 2281.56	valid_0's l2: 5.20549e+06
[19]	valid_0's rmse: 225

In [33]:
%time grid.best_estimator_.fit(features_train_ridge, target_train)
%time test_predicted_lgbm = model_2.predict(features_valid_ridge)
rmse_lgbm = math.sqrt(mean_squared_error(target_valid, test_predicted_lgbm))
print(rmse_lgbm)

[LightGBM] [Warning] Unknown parameter: score
[LightGBM] [Warning] Unknown parameter: return_train_score
CPU times: user 58.8 s, sys: 802 ms, total: 59.6 s
Wall time: 59.9 s
CPU times: user 175 ms, sys: 52.3 ms, total: 227 ms
Wall time: 213 ms
2226.441117256508


Значение метрики RMSE для **градиентного бустинга** получилось равным 2226.44, время обучения на валидационной выборке -  59.9 s и предсказания: 213 ms.

In [25]:
# трансформируем тестовую выборку
features_test_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.transform(features_test_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)


features_test_ridge.shape

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(28721, 348)

**Выводы**:
- Были построены две модели машинного обучения: линейная регрессия и градиентный бустинг;
- Значение метрики RMSE для **градиентного бустинга** получилось равным 2226.44, время обучения на валидационной выборке -  59.9 s и предсказания: 213 ms.
- Значение метрики RMSE для **линейной регрессии** получилось равным 1808.25, время обучения на валидационной выборке -  43.3 s, предсказания: 138 ms.
- Анализируя две модели, стоит сделать выбор в пользу **линейной регрессии**: она более качественная и быстробучаемая.

## 3. Анализ моделей

In [34]:
# Проверим модель на тестовой выборке

%time predicted_test = model.predict(features_test_ridge)
rmse_test = math.sqrt(mean_squared_error(target_test, predicted_test))
print(rmse_test)

CPU times: user 30.2 ms, sys: 0 ns, total: 30.2 ms
Wall time: 30.8 ms
1779.4449937669915


**Выводы**:
- Значение метрики RMSE для **линейной регрессии** на тествой выборке получилось равным 1779.44, время предсказания на тестовой выборке -  43.3 s\.